<a href="https://colab.research.google.com/github/dau-J/pypi_textminor/blob/master/textminer_pro_oss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# textminer-pro: 텍스트 전처리 분석

## 정현성

NLTK, scikit-learn, Sumy, langdetect

- 불용어 제거 (remove_stopwords)
- 키워드 추출 (extract_keywords)
- 텍스트 요약 (summarize_text)
- 언어 감지 (detect_language)

# 폴더 생성

In [2]:
!rm -rf textminer_pro
!mkdir -p textminer_pro/textminer
!mkdir -p textminer_pro/tests
!mkdir -p textminer_pro/.github/workflows

# 필요 라이브러리 설치

In [3]:
!pip install langdetect
!pip install sumy
!pip install twine

# cleaner.py

In [4]:
%%writefile textminer_pro/textminer/cleaner.py
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

def remove_stopwords(text: str, lang='english'):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words(lang))
    filtered = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered)


Writing textminer_pro/textminer/cleaner.py


# keyword.py

In [5]:
%%writefile textminer_pro/textminer/keyword.py
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_keywords(text: str, top_n=5):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text])
    scores = zip(vectorizer.get_feature_names_out(), tfidf_matrix.toarray()[0])
    sorted_words = sorted(scores, key=lambda x: x[1], reverse=True)
    return [word for word, score in sorted_words[:top_n]]

Writing textminer_pro/textminer/keyword.py


# summarizer.py

In [6]:
%%writefile textminer_pro/textminer/summarizer.py

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer


def summarize_text(text: str, ratio=0.2, language='english'):

    parser = PlaintextParser.from_string(text, Tokenizer(language))
    summarizer = LsaSummarizer()
    sentence_count = max(1, int(len(parser.document.sentences) * ratio))
    summary = summarizer(parser.document, sentence_count)
    return ' '.join(str(sentence) for sentence in summary)

Writing textminer_pro/textminer/summarizer.py


# detector.py

In [7]:
%%writefile textminer_pro/textminer/detector.py
from langdetect import detect

def detect_language(text: str):
    try:
        return detect(text)
    except:
        return "unknown"

Writing textminer_pro/textminer/detector.py


# __init__.py

In [8]:
%%writefile textminer_pro/textminer/__init__.py
from .cleaner import remove_stopwords
from .keyword import extract_keywords
from .summarizer import summarize_text
from .detector import detect_language

Writing textminer_pro/textminer/__init__.py


# README.md

In [9]:
%%writefile textminer_pro/README.md

# textminerpro

A Korean-friendly text preprocessing toolkit that supports:

- remove_stopwords(text, lang='en')
- extract_keywords(text, top_n=5)
- summarize_text(text, ratio=0.2)
- detect_language(text)

Writing textminer_pro/README.md


# setup.py

In [10]:
%%writefile textminer_pro/setup.py
from setuptools import setup, find_packages

setup(
    name='textminerpro-hyeonsung',
    version='0.0.1',
    author='정현성',
    author_email='2254784@donga.ac.kr',
    packages=find_packages(),
    install_requires=[
        'nltk',
        'scikit-learn',
        'sumy',
        'langdetect'
    ],
    description='Advanced text preprocessing package',
    long_description=open('README.md', encoding='utf-8').read(),
    long_description_content_type='text/markdown',
    url='https://github.com/dau-J/pypi_textminor.git',
    classifiers=[
        'Programming Language :: Python :: 3',
        'License :: OSI Approved :: MIT License',
        'Operating System :: OS Independent',
    ],
    python_requires='>=3.6',
)


Writing textminer_pro/setup.py


# test_cleaner.py

In [11]:
%%writefile textminer_pro/tests/test_cleaner.py
from textminer.cleaner import remove_stopwords
from nltk.tokenize import word_tokenize

def test_remove_stopwords():
    text = "This is a sample sentence"
    result = remove_stopwords(text)
    tokens = word_tokenize(result)

    assert "is" not in tokens
    assert "a" not in tokens
    assert "sample" in tokens
    assert "sentence" in tokens


Writing textminer_pro/tests/test_cleaner.py


# test_dector.py

In [12]:
%%writefile textminer_pro/tests/test_detector.py
from textminer.detector import detect_language

def test_detect_language():
    text = "This is an English sentence."
    lang = detect_language(text)
    assert lang == "en"


Writing textminer_pro/tests/test_detector.py


# bdist_wheel

In [13]:
%cd /content/textminer_pro
!python setup.py sdist bdist_wheel

/content/textminer_pro
running sdist
running egg_info
creating textminerpro_hyeonsung.egg-info
writing textminerpro_hyeonsung.egg-info/PKG-INFO
writing dependency_links to textminerpro_hyeonsung.egg-info/dependency_links.txt
writing requirements to textminerpro_hyeonsung.egg-info/requires.txt
writing top-level names to textminerpro_hyeonsung.egg-info/top_level.txt
writing manifest file 'textminerpro_hyeonsung.egg-info/SOURCES.txt'
reading manifest file 'textminerpro_hyeonsung.egg-info/SOURCES.txt'
writing manifest file 'textminerpro_hyeonsung.egg-info/SOURCES.txt'
running check
creating textminerpro_hyeonsung-0.0.1
creating textminerpro_hyeonsung-0.0.1/tests
creating textminerpro_hyeonsung-0.0.1/textminer
creating textminerpro_hyeonsung-0.0.1/textminerpro_hyeonsung.egg-info
copying files to textminerpro_hyeonsung-0.0.1...
copying README.md -> textminerpro_hyeonsung-0.0.1
copying setup.py -> textminerpro_hyeonsung-0.0.1
copying tests/test_cleaner.py -> textminerpro_hyeonsung-0.0.1/tests

# token-api를 사용해서 배포(toekn-api에 api-key)

In [14]:
!pip install twine
!twine upload -u __token__ -p "pypi token 자리입니다 " dist/*


Uploading distributions to https://upload.pypi.org/legacy/
Uploading textminerpro_hyeonsung-0.0.1-py3-none-any.whl
100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 kB • 00:00 • ?
Uploading textminerpro_hyeonsung-0.0.1.tar.gz
100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 kB • 00:00 • ?

View at:
https://pypi.org/project/textminerpro-hyeonsung/0.0.1/


# nltk download

In [15]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# pytest

In [16]:
!pip install .
!pytest tests

Processing /content/textminer_pro
  Preparing metadata (setup.py) ... done
  Created wheel for textminerpro-hyeonsung: filename=textminerpro_hyeonsung-0.0.1-py3-none-any.whl size=3158 sha256=732b0c3a7dffb4969ccbc1b23a5b3f144a4734f26c055e5d5d52b2a8db183930
  Stored in directory: /tmp/pip-ephem-wheel-cache-1l88noip/wheels/2a/d4/4e/7dd7c42528f76921e6532a65c2244e23707490a7d4ee6e166c
Successfully built textminerpro-hyeonsung
============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-8.3.5, pluggy-1.6.0
rootdir: /content/textminer_pro
plugins: langsmith-0.3.44, anyio-4.9.0, typeguard-4.4.2
collected 2 items                                                              

tests/test_cleaner.py .                                                  [ 50%]
tests/test_detector.py .                                                 [100%]

============================== 2 passed in 6.83s ===============================


# 배포한 라이브러리 다운로드

In [32]:
pip install -i https://test.pypi.org/simple/ textminerpro-hyeonsung

Looking in indexes: https://test.pypi.org/simple/


# 프로그램 동작 테스트

In [33]:
from textminer import remove_stopwords, extract_keywords, summarize_text, detect_language

# 테스트용 텍스트
text_ko = "oss 수업이 너무 즐거웠습니다 교수님 그동안 고생하셨습니다"
text_en = "The OSS class was really enjoyable. Thank you professor for your efforts."
# 한글
print("KOREAN")
print("불용어 제거:", remove_stopwords(text_ko))
print("키워드 추출:", extract_keywords(text_ko))
print("요약:", summarize_text(text_ko))
print("언어 감지:", detect_language(text_ko))

# 영어
print("\nENGLISH")
print("불용어 제거:", remove_stopwords(text_en))
print("키워드 추출:", extract_keywords(text_en))
print("요약:", summarize_text(text_en))
print("언어 감지:", detect_language(text_en))


KOREAN
불용어 제거: oss 수업이 너무 즐거웠습니다 교수님 그동안 고생하셨습니다
키워드 추출: ['oss', '고생하셨습니다', '교수님', '그동안', '너무']
요약: oss 수업이 너무 즐거웠습니다 교수님 그동안 고생하셨습니다
언어 감지: ko

ENGLISH
불용어 제거: OSS class really enjoyable . Thank professor efforts .
키워드 추출: ['class', 'efforts', 'enjoyable', 'oss', 'professor']
요약: Thank you professor for your efforts.
언어 감지: en


# githup release 및 pypi release 자동화 워크플로우 설정

In [37]:
%%writefile .github/workflows/release.yml
name: Build and Publish to PyPI + GitHub Release

on:
  push:
    tags:
      - 'v0.0.1'

jobs:
  deploy:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout code
      uses: actions/checkout@v3

    - name: Set up Python
      uses: actions/setup-python@v4
      with:
        python-version: '3.x'

    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install setuptools wheel twine

    - name: Build package
      run: |
        python setup.py sdist bdist_wheel

    - name: Publish to PyPI
      env:
        TWINE_USERNAME: __token__
        TWINE_PASSWORD: ${{ secrets.PYPI_API_TOKEN }}
      run: |
        twine upload dist/*

    - name: Create GitHub Release
      uses: softprops/action-gh-release@v1
      with:
        files: |
          dist/*.whl
          dist/*.tar.gz

Overwriting .github/workflows/release.yml


In [41]:
!git branch
!git branch -m gh-pages master
!git push -u origin master --force

* gh-pages
Enumerating objects: 52, done.
Counting objects: 100% (52/52), done.
Delta compression using up to 2 threads
Compressing objects: 100% (40/40), done.
Writing objects: 100% (51/51), 15.93 KiB | 1.99 MiB/s, done.
Total 51 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), done.
To https://github.com/dau-J/pypi_textminor.git
 + 9723352...3b57258 master -> master (forced update)
Branch 'master' set up to track remote branch 'master' from 'origin'.


In [42]:
!git add .
!git commit -m "Add GitHub Actions workflow for release"
!git push origin master

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean
Everything up-to-date


In [43]:
!git tag v0.0.1
!git push origin v0.0.1

Total 0 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/dau-J/pypi_textminor.git
 * [new tag]         v0.0.1 -> v0.0.1


# MKDOCS 문서 자동생성

In [44]:
!mkdir -p textminer_pro/docs

In [45]:
!pip install mkdocs

!mkdocs new docs-site

!mv docs-site/docs/* textminer_pro/docs/
!mv docs-site/mkdocs.yml textminer_pro/
!rm -r docs-site

INFO    -  Creating project directory: docs-site
INFO    -  Writing config file: docs-site/mkdocs.yml
INFO    -  Writing initial docs: docs-site/docs/index.md


In [46]:
%%writefile mkdocs.yml
site_name: textminer-pro
repo_url: https://github.com/dau-J/pypi_textminor

nav:
  - 홈: index.md
  - 사용법: usage.md

theme:
  name: readthedocs

Overwriting mkdocs.yml


In [47]:
!mkdir -p docs

In [48]:
%%writefile docs/index.md

# pypi_textminor
pypi 라이브러리 텍스트마이닝 프로그램

textminer-pro
A Korean-friendly text preprocessing toolkit that supports:

remove_stopwords(text, lang='en')
extract_keywords(text, top_n=5)
summarize_text(text, ratio=0.2)
detect_language(text)

## pypi패키지 링크 및 사진
pypi패키지 링크 [https://pypi.org/project/textminerpro-hyeonsung/]
--------------------------------------------------------------------------------------------
![image](https://github.com/user-attachments/assets/1d0adada-88e6-43da-8bb1-c1be6be1c624)




## 📁 프로젝트 폴더 구조
------------------------------------------------------------------------------------------------

```
textminer_pro/
├── textminer/
│   ├── __init__.py
│   ├── cleaner.py
│   ├── summarizer.py
│   └── detector.py
│
├── tests/
│   ├── test_cleaner.py
│   └── test_detector.py
│
├── setup.py
├── README.md
└── .github/
    └── workflows/
        └── pypi.yml
```



### 기능

* **텍스트 정제:** 불용어를 효율적으로 제거하고 다른 텍스트 정규화 작업을 수행합니다.
* **텍스트 요약:** 긴 텍스트의 간결한 요약을 생성합니다.
* **언어 감지:** 주어진 텍스트의 언어를 자동으로 식별합니다.

### 설치

pip를 통해 TextMiner Pro를 설치할 수 있습니다:

```bash
pip install textminerpro-hyeonsung==0.0.1

Overwriting docs/index.md


In [49]:
%%writefile docs/usage.md
# 사용예시

from textminer import remove_stopwords, extract_keywords, summarize_text, detect_language

text = "OSS 수업은 정말 즐거웠습니다."

# 불용어 제거
print(remove_stopwords(text, lang="english"))

# 키워드 추출
print(extract_keywords(text))

# 텍스트 요약
print(summarize_text(text))

# 언어 감지
print(detect_language(text))



Overwriting docs/usage.md


# Git 연동 및 Github pages 호스팅

In [53]:
!git branch
!git branch -m master gh-pages

* gh-pages


In [54]:
!git remote remove origin

In [55]:
# 1. Git 초기화
!git init

# 2. 사용자 정보 등록
!git config --global user.email "2254784@donga.ac.kr"
!git config --global user.name "dau-J"

# 3. remote 등록
!git remote add origin https://dau-J: github token 자리입니다 @github.com/dau-J/pypi_textminor.git

# 4. 변경 파일 add & commit
!git add .
!git commit -m "mkdocs deploy test"

# 5. github-pages 브랜치로 push
!git branch -M gh-pages
!git push -u origin

# 6. mkdocs 배포
!mkdocs gh-deploy --force

Reinitialized existing Git repository in /content/textminer_pro/.git/
[gh-pages 2b1df95] mkdocs deploy test
 1 file changed, 56 insertions(+), 53 deletions(-)
 rewrite docs/index.md (86%)
fatal: The current branch gh-pages has no upstream branch.
To push the current branch and set the remote as upstream, use

    git push --set-upstream origin gh-pages

INFO    -  Cleaning site directory
INFO    -  Building documentation to directory: /content/textminer_pro/site
INFO    -  Documentation built in 0.10 seconds
WARNING -  Version check skipped: No version specified in previous deployment.
INFO    -  Copying '/content/textminer_pro/site' to 'gh-pages' branch and pushing to GitHub.
Enumerating objects: 50, done.
Counting objects: 100% (50/50), done.
Delta compression using up to 2 threads
Compressing objects: 100% (44/44), done.
Writing objects: 100% (47/47), 2.67 MiB | 4.14 MiB/s, done.
Total 47 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), completed wi

In [57]:
!git branch
!git branch -m gh-pages master

* master
fatal: A branch named 'master' already exists.


# 폴더 압축 및 다운로드

In [30]:
# 전체 폴더 압축 (예: textminer_pro)
import shutil
shutil.make_archive('/content/textminer_pro', 'zip', '/content/textminer_pro')

'/content/textminer_pro.zip'

In [31]:
from google.colab import files
files.download('/content/textminer_pro.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>